In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pandas import DataFrame, Series
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler

from datatools import extract_market_data, data_quantization
from pipeline import Dataset
from pipeline.backtest import evaluation_for_submission, cross_validation
from visualization.metric import plot_performance

import xarray as xr

In [3]:
from qids_lib import QIDS

dataset = Dataset.load('../../data/parsed')
qids = QIDS(path_prefix='../../')

In [4]:
from datatools import calculate_market_return

m_df = extract_market_data(dataset.market)
market_return = calculate_market_return(m_df)
full_df = pd.concat([dataset.fundamental, m_df, dataset.ref_return], axis=1).merge(market_return, left_on='day', right_index=True).dropna()

In [7]:
ds = xr.Dataset.from_dataframe(full_df)

In [8]:
ds

<xarray.Dataset>
Dimensions:            (day: 998, asset: 54)
Coordinates:
  * day                (day) int64 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998
  * asset              (asset) int64 0 1 2 3 4 5 6 7 ... 46 47 48 49 50 51 52 53
Data variables: (12/13)
    turnoverRatio      (day, asset) float64 3.679 2.515 1.286 ... 0.5684 1.293
    transactionAmount  (day, asset) float64 1.723e+04 3.706e+03 ... 9.681e+03
    pe_ttm             (day, asset) float64 34.44 28.99 42.94 ... 28.99 10.65
    pe                 (day, asset) float64 32.3 27.27 41.93 ... 28.99 10.65
    pb                 (day, asset) float64 4.942 5.055 4.808 ... 5.785 1.996
    ps                 (day, asset) float64 3.818 3.048 4.139 ... 3.715 0.4073
    ...                 ...
    avg_price          (day, asset) float64 24.55 16.17 9.059 ... 28.85 9.821
    volatility         (day, asset) float64 0.862 0.8046 ... 0.8039 0.5421
    mean_volume        (day, asset) float64 4.541e+05 1.769e+05 ... 2.528e+05
    return_0           (day, asset) float64 -0.02523 -0.0119 ... 0.01992
    return             (day, asset) float64 -0.02688 -0.05267 ... -0.003662
    market_return_0    (day, asset) float64 -0.006928 -0.006928 ... -0.007671

In [67]:
ds.sel(day=slice(1, 10))[['pe', 'pb']].as_numpy().to_array(dim='feature')

<xarray.DataArray (feature: 2, day: 10, asset: 54)>
array([[[32.3029, 27.2726, 41.9279, ..., 36.3407, 44.321 , 14.8188],
        [31.2498, 25.5836, 40.3256, ..., 34.951 , 44.4425, 14.3498],
        [23.3887, 28.0593, 38.896 , ..., 54.3874, 50.1796, 20.4745],
        ...,
        [24.9441, 30.6429, 39.923 , ..., 54.2797, 51.8377, 21.0385],
        [25.405 , 31.3489, 39.7574, ..., 52.1258, 51.8649, 20.7687],
        [25.3474, 33.2716, 40.2709, ..., 52.1258, 52.5988, 21.0139]],

       [[ 4.9425,  5.0552,  4.8083, ...,  2.4076,  2.6026,  4.0209],
        [ 4.7814,  4.7422,  4.6246, ...,  2.3155,  2.6097,  3.8936],
        [ 4.3823,  4.6361,  4.617 , ...,  2.405 ,  2.617 ,  3.9813],
        ...,
        [ 4.6737,  5.0629,  4.7389, ...,  2.4003,  2.7034,  4.0909],
        [ 4.7601,  5.1796,  4.7193, ...,  2.305 ,  2.7049,  4.0385],
        [ 4.7493,  5.4973,  4.7802, ...,  2.305 ,  2.7431,  4.0862]]])
Coordinates:
  * day      (day) int64 1 2 3 4 5 6 7 8 9 10
  * asset    (asset) int64 0 1 2 3 4 5 6 7 8 9 ... 44 45 46 47 48 49 50 51 52 53
  * feature  (feature) object 'pe' 'pb'

In [77]:
start_day = 100
i = 1
per_eval_window = 10
ds.sel(day=slice(100, 120)).sel(day=slice(start_day+i, start_day+i+per_eval_window)).to_array(dim='feature').transpose('feature', 'day', 'asset').to_numpy()

array([[[ 6.24770000e+00,  2.48960000e+00,  2.73700000e+00, ...,
          6.77730000e+00,  6.06540000e+00,  2.82580000e+00],
        [ 5.85700000e+00,  1.57200000e+00,  2.80400000e+00, ...,
          5.20190000e+00,  5.65380000e+00,  2.34310000e+00],
        [ 6.81720000e+00,  1.59970000e+00,  3.12010000e+00, ...,
          5.77850000e+00,  4.71860000e+00,  2.72720000e+00],
        ...,
        [ 5.90690000e+00,  2.38670000e+00,  2.13370000e+00, ...,
          6.38290000e+00,  3.43380000e+00,  2.18580000e+00],
        [ 5.33110000e+00,  2.08430000e+00,  2.72030000e+00, ...,
          7.65920000e+00,  2.78700000e+00,  3.16020000e+00],
        [ 6.07120000e+00,  2.20280000e+00,  2.38170000e+00, ...,
          9.79930000e+00,  3.80160000e+00,  3.83970000e+00]],

       [[ 3.33170000e+04,  6.59900000e+03,  1.60890000e+04, ...,
          4.31020000e+04,  1.86270000e+04,  1.23720000e+04],
        [ 2.83980000e+04,  6.46300000e+03,  1.31120000e+04, ...,
          3.28470000e+04,  1.43980000e

In [79]:
ds.day.min().to_numpy()

array(1, dtype=int64)